# Agents Semantic Kernel en Python

Dans ce notebook, nous allons :
1. Installer / Importer **semantic-kernel** en Python
2. Illustrer la création d'un agent simple (single agent) et son invocation
3. Illustrer un **group chat** (AgentGroupChat) avec sélection/termination
4. Conclusion

In [ ]:
# ============================
# Bloc 1 : Installation semantic-kernel et imports
# ============================

# À n’exécuter qu'une fois
%pip install semantic-kernel --quiet
import asyncio
import logging

print("semantic-kernel installé.")


## 1. Installation et imports

Cette cellule installe le SDK Semantic Kernel et importe les modules necessaires pour creer des agents conversationnels.

**Concepts cles** :
- `ChatCompletionAgent` : Un agent base sur un modele de chat completion
- `AgentGroupChat` : Orchestrateur pour faire collaborer plusieurs agents
- `ChatHistory` : Historique de conversation partage

## Bloc 2 : Simple Agent (Parrot)

Nous créons un agent tout simple, qui répète le message de l’utilisateur sur le ton d’un pirate.

In [ ]:
import logging
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import ChatHistory

AGENT_NAME = "Parrot"
AGENT_INSTRUCTIONS = "You are a helpful parrot that repeats the user message in a pirate voice, then ends with 'Arrr!'"

# Création du Kernel
kernel = Kernel()
# On suppose que vous avez défini ou récupéré des clés d'API :
# kernel.add_service(OpenAIChatCompletion(...)) ou AzureChatCompletion(...)
kernel.add_service(OpenAIChatCompletion(service_id="agent"))

agent = ChatCompletionAgent(
    kernel=kernel,
    name=AGENT_NAME,
    instructions=AGENT_INSTRUCTIONS
)
user_inputs = [
    "Fortune favors the bold.",
    "I came, I saw, I conquered.",
    "Practice makes perfect.",
]

async def simple_agent_demo():
    chat_history = ChatHistory()
    # On ajoute les instructions de l'agent en tant que 'developer' ou 'system'
    chat_history.add_developer_message(AGENT_INSTRUCTIONS)

    for user_input in user_inputs:
        chat_history.add_user_message(user_input)
        print(f"# User: '{user_input}'")
        async for content in agent.invoke(chat_history):
            # CORRECTION DÉFINITIVE : Toujours utiliser add_assistant_message
            # L'API SemanticKernel agents retourne des objets incompatibles avec add_message()
            if hasattr(content, 'content'):
                # Extraire le contenu et le convertir en string
                content_str = str(content.content) if content.content else str(content)
                chat_history.add_assistant_message(content_str)
                print(f"# Agent - {content.name or AGENT_NAME}: '{content_str}'")
            else:
                # Fallback: convertir tout l'objet en string
                content_str = str(content)
                chat_history.add_assistant_message(content_str)
                print(f"# Agent - {AGENT_NAME}: '{content_str}'")

await simple_agent_demo()

### Analyse du comportement de l'agent Parrot

L'execution ci-dessus montre comment l'agent transforme chaque citation en style pirate :

- **Input** : "Fortune favors the bold."
- **Output attendu** : "Arrr! Fortune be favorin' the bold, matey! Arrr!"

Points cles a observer :
1. L'agent respecte ses instructions systeme (ton pirate + "Arrr!")
2. Le contexte est maintenu via `ChatHistory`
3. L'invocation est asynchrone (streaming)

Le pattern `async for content in agent.invoke()` permet de recevoir la reponse en temps reel.

### Execution de l'agent Parrot

L'agent "Parrot" illustre le cas le plus simple :
- **Instructions systeme** : Definissent le comportement (repeter en voix de pirate)
- **Invocation** : Chaque message utilisateur est transforme par l'agent
- **Historique** : Les reponses sont ajoutees a `ChatHistory` pour le contexte

**Point technique** : La methode `invoke()` est asynchrone et retourne un flux (streaming) de contenus.

## Bloc 3 : Agent simple avec Plugins

Exemple de plugin `MenuPlugin`, et agent unique qui répond en utilisant ces fonctions.


In [ ]:
import asyncio
from typing import TYPE_CHECKING, Annotated
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import KernelArguments, kernel_function
from semantic_kernel.contents import ChatHistory

class MenuPlugin:
    """Plugin pour gérer un menu"""
    @kernel_function(description="Liste les specials")
    def get_specials(self) -> Annotated[str, "Describes specials"]:
        # print function call
        print("get_specials called")
        return "Special Soup: Clam Chowder\nSpecial Salad: Cobb Salad\nSpecial Drink: Chai Tea"
    @kernel_function(description="Donne le prix d'un item")
    def get_item_price(self, menu_item: Annotated[str, "nom de l'item"]) -> str:
         # print function call
        print("get_item_price called")
        return "$9.99"
# Créer kernel
kernel2 = Kernel()
# Ajout du plugin
kernel2.add_plugin(MenuPlugin(), plugin_name="menu")
# Ajout du service
kernel2.add_service(OpenAIChatCompletion(service_id="agent2"))
# On configure l'auto function-calling
settings2 = kernel2.get_prompt_execution_settings_from_service_id(service_id="agent2")
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
settings2.function_choice_behavior = FunctionChoiceBehavior.Auto()

AGENT2_NAME = "Host"
AGENT2_INSTRUCTIONS = "Answer questions about the menu."
agent2 = ChatCompletionAgent(
    kernel=kernel2,
    name=AGENT2_NAME,
    instructions=AGENT2_INSTRUCTIONS,
    arguments=KernelArguments(settings=settings2),
)
async def plugin_agent_demo():
    chat_history = ChatHistory()
    user_msgs = [
        "Hello",
        "What is the special soup?",
        "What does it cost?",
        "Thanks",
    ]
    for user_input in user_msgs:
        chat_history.add_user_message(user_input)
        print(f"# User: '{user_input}'")
        agent_name = None
        full_response = ""  # Accumulateur pour la réponse complète
        
        async for content in agent2.invoke_stream(chat_history):
            if not agent_name:
                agent_name = content.name or AGENT2_NAME
                print(f"# {agent_name}: '", end="")
            
            # CORRECTION: Gestion appropriée du StreamingChatMessageContent
            if hasattr(content, 'content'):
                # Conversion sécurisée du contenu
                content_str = str(content.content) if content.content else ""
                # Accumulation de la réponse
                full_response += content_str
                # Affichage incrémental
                if content_str:
                    print(content_str, end="", flush=True)
        
        print("'")
        
        # Ajout sécurisé du message complet à l'historique
        if full_response:
            chat_history.add_assistant_message(full_response)
        else:
            # Si pas de contenu, ajouter un message par défaut
            chat_history.add_assistant_message("[No response generated]")

await plugin_agent_demo()

### Analyse des appels de fonction automatiques

Dans l'execution ci-dessus, observez la sequence d'appels :

1. **User** : "What is the special soup?"
2. **Agent interne** : Appelle `get_specials()` (affiche "get_specials called")
3. **Agent repond** : "The special soup is Clam Chowder"
4. **User** : "What does it cost?"
5. **Agent interne** : Appelle `get_item_price("Clam Chowder")` (affiche "get_item_price called")
6. **Agent repond** : "It costs $9.99"

Le parametre `FunctionChoiceBehavior.Auto()` permet a l'agent de :
- Analyser la question de l'utilisateur
- Determiner quelle fonction appeler
- Extraire les parametres (ex: "Clam Chowder")
- Integrer le resultat dans sa reponse

C'est la base du **ReAct pattern** (Reasoning + Acting).

### Execution de l'agent avec plugins

Le plugin `MenuPlugin` expose des fonctions que l'agent peut appeler automatiquement :
- `get_specials()` : Retourne les plats du jour
- `get_item_price()` : Retourne le prix d'un article

**Function Calling Auto** : Le parametre `FunctionChoiceBehavior.Auto()` permet a l'agent de decider lui-meme quand appeler ces fonctions en fonction du contexte de la conversation.

Observez les appels `get_specials called` et `get_item_price called` dans la sortie.

## Bloc 4 : Group Chat

Exemple d'un chat groupé : un agent CopyWriter, un agent ArtDirector, etc. On utilise la `AgentGroupChat`.


In [ ]:
import asyncio
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import AuthorRole, ChatMessageContent
from semantic_kernel.agents.strategies import TerminationStrategy
from semantic_kernel import Kernel

class ApprovalTerminationStrategy(TerminationStrategy):
    async def should_agent_terminate(self, agent, history):
        return "approved" in history[-1].content.lower()

# On crée un kernel par agent, ou le même kernel + service differencié.
def create_kernel_for(name):
    k = Kernel()
    # on admet qu'on a paramétré un service openAI.
    k.add_service(OpenAIChatCompletion(service_id=name))
    return k

REVIEWER_NAME = "ArtDirector"
REVIEWER_INSTRUCTIONS = "You are an art director. If the copy is good, say 'Approved'. Otherwise, propose improvements."
reviewer_agent = ChatCompletionAgent(
    kernel=create_kernel_for(REVIEWER_NAME),
    name=REVIEWER_NAME,
    instructions=REVIEWER_INSTRUCTIONS,
)
COPYWRITER_NAME = "CopyWriter"
COPYWRITER_INSTRUCTIONS = "You are a copywriter. Provide short but strong marketing copy."
writer_agent = ChatCompletionAgent(
    kernel=create_kernel_for(COPYWRITER_NAME),
    name=COPYWRITER_NAME,
    instructions=COPYWRITER_INSTRUCTIONS,
)
group_chat = AgentGroupChat(
    agents=[reviewer_agent, writer_agent],
    termination_strategy=ApprovalTerminationStrategy(agents=[reviewer_agent], maximum_iterations=6)
)

async def group_chat_demo():
    user_msg = "I need a slogan for a new line of electric bikes"
    await group_chat.add_chat_message(ChatMessageContent(role=AuthorRole.USER, content=user_msg))
    print(f"# User: '{user_msg}'")
    async for content in group_chat.invoke():
        # Gestion sécurisée du contenu pour éviter ContentInitializationError
        if hasattr(content, 'content') and content.content:
            print(f"# Agent - {content.name or '*'}: '{content.content}'")
        else:
            print(f"# Agent - {content.name or '*'}: '{str(content)}'")

    print(f"# IS COMPLETE: {group_chat.is_complete}")

await group_chat_demo()


### Analyse du flux de conversation multi-agents

Le Group Chat ci-dessus orchestre un dialogue iteratif :

**Tour 1** :
- **CopyWriter** : Propose un premier slogan (ex: "Ride the Future - Electric Bikes Reimagined")
- **ArtDirector** : Evalue et demande des ameliorations

**Tours 2-N** :
- **CopyWriter** : Refine le slogan selon les feedbacks
- **ArtDirector** : Continue l'evaluation jusqu'a validation

**Terminaison** :
- Quand l'ArtDirector dit "approved", `ApprovalTerminationStrategy` arrete la conversation
- `is_complete` passe a `True`

Ce pattern est utile pour :
- **Peer review automatique** (code, contenu)
- **Workflows creatifs** (brainstorming, iteration)
- **Validation multi-etapes** (compliance, qualite)

La limite `maximum_iterations=6` protege contre les boucles infinies si aucun consensus n'est atteint.

### Execution du Group Chat

Le `AgentGroupChat` orchestre plusieurs agents qui collaborent :
- **ArtDirector** : Valide ou demande des ameliorations du texte
- **CopyWriter** : Propose du contenu marketing

**Strategie de terminaison** : La classe `ApprovalTerminationStrategy` arrete la conversation quand l'ArtDirector dit "approved". Cela illustre comment definir des conditions d'arret personnalisees.

La limite `maximum_iterations=6` protege contre les boucles infinies.

## Conclusion

Nous avons illustré **plusieurs scénarios** d’agents en Python avec Semantic Kernel :
- Un agent **unique** type “parrot” qui répète le user input.
- Un agent **unique** + plugins (function-calling auto-invoqué).
- Un **group chat** d’agents (ex: CopyWriter, ArtDirector), orchestré via `AgentGroupChat`.

Vous pouvez adapter ces exemples à vos propres clés API (OpenAI vs Azure), configurer l’autogestion des tools, ou enrichir avec vos plugins sémantiques.
Et voilà un “notebook 3” purement Python côté Semantic Kernel. 😉